In [117]:
import os
import requests
import csv
import pandas as pd

In [118]:
#change these
filename = "21-202rw"
html_ = f"{filename}.html"
csv_ =  f"{filename}.csv"
meet = "2021 PV Maryland State LC Championship - 6/4/2021 to 6/6/2021"

In [119]:
# Path
data_folder = "data"
 
# Join various path components
html_file = os.path.join(data_folder, html_)
print(html_file)

data\21-202rw.html


In [120]:
#Creating an output file in writing mode
csv_file = os.path.join(data_folder,csv_)

#csv header
header = ["meet", "event", "rank", "name", "age", "team", "seed", "finals"]
#assign 'delete' event for delition later
event = "delete"

with open(csv_file, 'w', encoding='UTF8') as f:
    #create a writer
    writer = csv.writer(f, delimiter = "|")
    # write the header
    writer.writerow(header)
    #iterate html
    with open(html_file, "r") as line:
        line_string = line.readlines()
        #this will assign event column
        for each_line in line_string:
            if each_line[0:2] == "<b":
                event = each_line.replace("<b>", "")\
                                 .replace("</b>", "")\
                                 .replace("&amp;", "&")\
                                 .replace("\n", " ")\
                                 .rstrip()
            else:
                #assign each column and strip right spaces
                rank = each_line[0:4].replace(" ", "").replace("=", "").rstrip()
                name = each_line[4:29].rstrip()
                age = each_line[28:33].replace(" ","").rstrip()
                team = each_line[33:50].replace(" ","").rstrip()
                seed = each_line[51:65].replace(" ","").rstrip()
                finals = each_line[61:80].replace(" ","").rstrip()
                #this will not write uncessary rows
                if rank != "" and event != 'delete'\
                   and seed != "" and finals != "":
                    writer.writerow([meet, event, rank, name, age, team, seed, finals])



In [121]:
#read csv to data frame
df = pd.read_csv(csv_file, sep="|"  , engine="python")

In [122]:
#filtering examples
filt = (df["name"] == "Kertajaya, Evelyn") &  (df["rank"] == "12")

In [123]:
#get data
df[filt]

,meet,event,rank,name,age,team,seed,finals
326,2021 PV Maryland State LC Championship - 6/4/2...,Girls 11-12 50 LC Meter Breaststroke,12,"Kertajaya, Evelyn",12,RMSC!-PV,43.68,42.75


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1151 entries, 0 to 1150
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   meet    1151 non-null   object
 1   event   1151 non-null   object
 2   rank    1151 non-null   object
 3   name    1151 non-null   object
 4   age     1151 non-null   int64 
 5   team    1151 non-null   object
 6   seed    1151 non-null   object
 7   finals  1151 non-null   object
dtypes: int64(1), object(7)
memory usage: 72.1+ KB


In [125]:
# with open(html_file, "r") as line:
#    line_string = line.readlines()
#    for each_line in line_string:
#       print(each_line[0:30])

In [126]:
# from bs4 import BeautifulSoup
# # Opening the html file
# with open(html_file) as fp:
#     soup = BeautifulSoup(fp, "html.parser")

In [127]:
# #get all events and create a list
# event_list = []
# events = soup.find_all("b")
# for event in events:
#     #(type(event.text))
#     event = event.text.replace("<b>", "").replace("</b>", "")
#     event_list.append(event)



In [128]:
# string = "  1 Mostofi, Addison B        14 ASA-PV               4:46.41    4:45.45  "

In [129]:
# name = string[0:30]
# print(name)

# age = string[29:33].replace(" ","")
# print(age)

# team = string[33:50].replace(" ","")
# print(team)

# seed = string[51:65].replace(" ","")
# print(seed)

# finals = string[61:80].replace(" ","")
# print(finals)
